# Ejercicio 04: Evaluación de un Sistema de Recuperación de Información


## 1. Proporcionar un conjunto de datos

In [1]:
import xml.etree.ElementTree as ET
import numpy as np

In [2]:
# Cargar y parsear el archivo XML
file_path = '/kaggle/input/03ranking-corpus/03ranking_corpus.xml'
tree = ET.parse(file_path)
root = tree.getroot()

In [3]:
# Crear una lista para almacenar los documentos con sus metadatos
documents = []

# Iterar a través de cada elemento 'document' en el archivo XML
for doc in root.findall('document'):
    doc_id = doc.get('id')
    title = doc.find('title').text
    keywords = doc.find('keywords').text
    author = doc.find('author').text
    date = doc.find('date').text
    
    documents.append({
        'id': doc_id,
        'title': title,
        'keywords': keywords,
        'author': author,
        'date': date
    })

In [4]:
# Definir las consultas
queries = {
    1: "Impacto de la salud mental en el rendimiento académico de los estudiantes universitarios",
    2: "Actividades extracurriculares y bienestar emocional en el campus universitario",
    3: "Estrategias universitarias para reducir el estrés en estudiantes"
}

In [22]:
def calculate_relevance(query, document):
    """
    Calcula el nivel de coincidencia entre la consulta y las palabras clave del documento.
    """
    query_words = set(query.lower().split())
    document_words = set(document['keywords'].lower().split())
    common_words = query_words & document_words
    return len(common_words)


In [23]:
# Umbral mínimo de coincidencias para considerar un documento relevante
threshold = 2

# Diccionario para almacenar los documentos relevantes por consulta
relevant_docs_per_query = {}

for query_id, query_text in queries.items():
    relevant_docs = []
    for document in documents:
        relevance_score = calculate_relevance(query_text, document)
        if relevance_score >= threshold:  # Consideramos relevante si la coincidencia es igual o superior al umbral
            relevant_docs.append(document['id'])
    relevant_docs_per_query[query_id] = relevant_docs


In [24]:
for query_id, relevant_docs in relevant_docs_per_query.items():
    print(f"Consulta {query_id} - Documentos relevantes: {relevant_docs}")


Consulta 1 - Documentos relevantes: ['2', '7', '11', '12', '13', '14', '18', '19', '29']
Consulta 2 - Documentos relevantes: ['15', '25']
Consulta 3 - Documentos relevantes: ['11', '29']


## 2. Calcular resultados de busqueda

In [25]:
# Función para procesar el texto y extraer palabras clave
def process_text(text):
    text = text.lower()
    import re
    text = re.sub(r'[^a-záéíóúñü]+', ' ', text)
    tokens = text.strip().split()
    return set(tokens)

# Función de similitud coseno
def cosine_similarity(query, doc):
    query_tokens = process_text(query)
    doc_tokens = process_text(doc)
    vocabulary = list(query_tokens | doc_tokens)
    query_vector = [1 if word in query_tokens else 0 for word in vocabulary]
    doc_vector = [1 if word in doc_tokens else 0 for word in vocabulary]
    dot_product = np.dot(query_vector, doc_vector)
    norm_query = np.linalg.norm(query_vector)
    norm_doc = np.linalg.norm(doc_vector)
    return dot_product / (norm_query * norm_doc) if norm_query != 0 and norm_doc != 0 else 0

# Función de similitud de Jaccard
def jaccard_similarity(query, doc):
    query_tokens = process_text(query)
    doc_tokens = process_text(doc)
    intersection = query_tokens & doc_tokens
    union = query_tokens | doc_tokens
    return len(intersection) / len(union)


In [26]:
# Número de documentos a mostrar
k = 3

# Función para rankear los documentos
def rank_documents(queries, similarity_function):
    rankings = {}
    for query_id, query_text in queries.items():
        similarity_scores = []
        for doc in documents:
            doc_text = doc['title'] + ' ' + doc['keywords']
            similarity_score = similarity_function(query_text, doc_text)
            similarity_scores.append((doc['id'], doc['title'], similarity_score))
        ranked_docs = sorted(similarity_scores, key=lambda x: x[2], reverse=True)
        rankings[query_id] = ranked_docs[:k]
    return rankings

# Rankear los documentos usando la similitud coseno
cosine_rankings = rank_documents(queries, cosine_similarity)
print("Ranking por Similitud Coseno:")
for query_id, ranked_docs in cosine_rankings.items():
    print(f"Consulta {query_id}:")
    for rank, (doc_id, doc_title, score) in enumerate(ranked_docs, start=1):
        print(f" {rank}. Documento {doc_id} - (Similitud Coseno: {score:.4f})")
        print(f" {doc_title}")
    print("\n")

# Rankear los documentos usando la similitud de Jaccard
jaccard_rankings = rank_documents(queries, jaccard_similarity)
print("Ranking por Similitud de Jaccard:")
for query_id, ranked_docs in jaccard_rankings.items():
    print(f"Consulta {query_id}:")
    for rank, (doc_id, doc_title, score) in enumerate(ranked_docs, start=1):
        print(f" {rank}. Documento {doc_id} - (Similitud Jaccard: {score:.4f})")
        print(f" {doc_title}")
    print("\n")


Ranking por Similitud Coseno:
Consulta 1:
 1. Documento 2 - (Similitud Coseno: 0.6405)
 Cómo la nutrición balanceada afecta el rendimiento académico y la salud mental en estudiantes.
 2. Documento 7 - (Similitud Coseno: 0.6405)
 La importancia del sueño en la salud mental y el rendimiento académico en jóvenes universitarios.
 3. Documento 14 - (Similitud Coseno: 0.6301)
 Cómo el acceso a servicios de salud mental en la universidad puede mejorar el desempeño académico.


Consulta 2:
 1. Documento 15 - (Similitud Coseno: 0.5547)
 Beneficios de las actividades extracurriculares para el bienestar emocional y social de los estudiantes.
 2. Documento 25 - (Similitud Coseno: 0.4623)
 La relación entre actividades artísticas y el bienestar psicológico en la vida universitaria.
 3. Documento 8 - (Similitud Coseno: 0.4303)
 La influencia del apoyo familiar y social en el bienestar emocional de los estudiantes universitarios.


Consulta 3:
 1. Documento 19 - (Similitud Coseno: 0.4725)
 Factores d

## 3. Calcular las métricas de evaluación

### Precisión en el top-k 

In [27]:
def precision_at_k(retrieved_docs, relevant_docs, k):
    """
    Calcula la precisión en el top-k (Prec@k).
    """
    retrieved_k = retrieved_docs[:k]  # Documentos en el top-k
    relevant_retrieved = sum(1 for doc_id in retrieved_k if doc_id in relevant_docs)  # Documentos relevantes en el top-k
    return relevant_retrieved / k  # Precisión en el top-k


### Recall

In [28]:
def recall(retrieved_docs, relevant_docs):
    """
    Calcula el Recall para una consulta dada.
    """
    relevant_retrieved = sum(1 for doc_id in retrieved_docs if doc_id in relevant_docs)
    return relevant_retrieved / len(relevant_docs) if relevant_docs else 0


### F1- score

In [29]:
def f1_score(precision, recall):
    """
    Calcula el F1-score dado la precisión y el recall.
    """
    if precision + recall == 0:
        return 0
    return 2 * (precision * recall) / (precision + recall)


### MAP

In [30]:
def average_precision(retrieved_docs, relevant_docs):
    """
    Calcula la Average Precision (AP) para una consulta.
    """
    precisions = []
    relevant_retrieved = 0
    
    for k, doc_id in enumerate(retrieved_docs, start=1):
        if doc_id in relevant_docs:
            relevant_retrieved += 1
            precisions.append(relevant_retrieved / k)  # Prec@k para este punto
    
    return sum(precisions) / len(relevant_docs) if relevant_docs else 0


In [31]:
def mean_average_precision(ranked_docs_per_query, relevant_docs_per_query):
    """
    Calcula el Mean Average Precision (MAP) para un sistema de recuperación.
    """
    average_precisions = []
    
    for query_id, retrieved_docs in ranked_docs_per_query.items():
        relevant_docs = relevant_docs_per_query[query_id]
        ap = average_precision([doc[0] for doc in retrieved_docs], relevant_docs)  # Extraer IDs de documentos recuperados
        average_precisions.append(ap)
    
    return sum(average_precisions) / len(average_precisions) if average_precisions else 0


### nDCG

In [32]:
import math

def dcg(retrieved_docs, relevant_docs, p):
    """
    Calcula el Discounted Cumulative Gain (DCG) hasta la posición p.
    """
    dcg_value = 0
    for i, doc_id in enumerate(retrieved_docs[:p]):
        relevance = 1 if doc_id in relevant_docs else 0
        dcg_value += relevance / math.log2(i + 2)  # i+2 porque log2(1) es 0
    return dcg_value

def idcg(relevant_docs, p):
    """
    Calcula el Ideal Discounted Cumulative Gain (IDCG) hasta la posición p.
    """
    ideal_relevances = [1] * min(len(relevant_docs), p)
    return sum(rel / math.log2(i + 2) for i, rel in enumerate(ideal_relevances))

def ndcg(retrieved_docs, relevant_docs, p):
    """
    Calcula el Normalized Discounted Cumulative Gain (nDCG) hasta la posición p.
    """
    dcg_value = dcg(retrieved_docs, relevant_docs, p)
    idcg_value = idcg(relevant_docs, p)
    return dcg_value / idcg_value if idcg_value != 0 else 0


### Resultado de las métricas de evaluación

In [33]:
# Parámetros
k = 3  # Top-k para Prec@k y nDCG

# Resultados para Similitud Coseno
print("Resultados para Similitud Coseno:")
for query_id, ranked_docs in cosine_rankings.items():
    retrieved_docs = [doc[0] for doc in ranked_docs]
    relevant_docs = relevant_docs_per_query[query_id]

    # Calcular Prec@k, Recall, F1-score, MAP, nDCG
    precision = precision_at_k(retrieved_docs, relevant_docs, k)
    recall_value = recall(retrieved_docs, relevant_docs)
    f1 = f1_score(precision, recall_value)
    ndcg_value = ndcg(retrieved_docs, relevant_docs, k)
    ap = average_precision(retrieved_docs, relevant_docs)
    
    print(f"Consulta {query_id}:")
    print(f" Prec@{k} = {precision:.4f}")
    print(f" Recall = {recall_value:.4f}")
    print(f" F1-score = {f1:.4f}")
    print(f" nDCG@{k} = {ndcg_value:.4f}")
    print(f" AP = {ap:.4f}")
    print("\n")

# Mean Average Precision (MAP) para Similitud Coseno
cosine_map = mean_average_precision(cosine_rankings, relevant_docs_per_query)
print(f"MAP para Similitud Coseno: {cosine_map:.4f}")

# Resultados para Similitud de Jaccard
print("\nResultados para Similitud de Jaccard:")
for query_id, ranked_docs in jaccard_rankings.items():
    retrieved_docs = [doc[0] for doc in ranked_docs]
    relevant_docs = relevant_docs_per_query[query_id]

    # Calcular Prec@k, Recall, F1-score, MAP, nDCG
    precision = precision_at_k(retrieved_docs, relevant_docs, k)
    recall_value = recall(retrieved_docs, relevant_docs)
    f1 = f1_score(precision, recall_value)
    ndcg_value = ndcg(retrieved_docs, relevant_docs, k)
    ap = average_precision(retrieved_docs, relevant_docs)
    
    print(f"Consulta {query_id}:")
    print(f" Prec@{k} = {precision:.4f}")
    print(f" Recall = {recall_value:.4f}")
    print(f" F1-score = {f1:.4f}")
    print(f" nDCG@{k} = {ndcg_value:.4f}")
    print(f" AP = {ap:.4f}")
    print("\n")

# Mean Average Precision (MAP) para Similitud de Jaccard
jaccard_map = mean_average_precision(jaccard_rankings, relevant_docs_per_query)
print(f"MAP para Similitud de Jaccard: {jaccard_map:.4f}")


Resultados para Similitud Coseno:
Consulta 1:
 Prec@3 = 1.0000
 Recall = 0.3333
 F1-score = 0.5000
 nDCG@3 = 1.0000
 AP = 0.3333


Consulta 2:
 Prec@3 = 0.6667
 Recall = 1.0000
 F1-score = 0.8000
 nDCG@3 = 1.0000
 AP = 1.0000


Consulta 3:
 Prec@3 = 0.0000
 Recall = 0.0000
 F1-score = 0.0000
 nDCG@3 = 0.0000
 AP = 0.0000


MAP para Similitud Coseno: 0.4444

Resultados para Similitud de Jaccard:
Consulta 1:
 Prec@3 = 1.0000
 Recall = 0.3333
 F1-score = 0.5000
 nDCG@3 = 1.0000
 AP = 0.3333


Consulta 2:
 Prec@3 = 0.6667
 Recall = 1.0000
 F1-score = 0.8000
 nDCG@3 = 1.0000
 AP = 1.0000


Consulta 3:
 Prec@3 = 0.0000
 Recall = 0.0000
 F1-score = 0.0000
 nDCG@3 = 0.0000
 AP = 0.0000


MAP para Similitud de Jaccard: 0.4444


## 4. Análisis y Comparación

- Comparar los resultados de los dos sistemas utilizando las métricas calculadas.

Ambos sistemas obtuvieron el mismo rendimiento debido a la estructura del conjunto de datos y las consultas. Aqui tambien juega un papel importante el tamaño del conjunto de documentos y consultas, ya que con 30 documentos y solo 3 consultas no hay tanta diversidad, lo que podria ser una razon para que se obtengan resultados similares.

- Discutir cuál sistema es más efectivo y por qué.
En este caso, ejercicio en especifico, no se podria decir que uno es mas efectivo que el otro, ya que presentan el mismo desempeño en toas las metricas calculadas. Para diferenciarlos mejor seria necesario aplicar un conjunto de datos mas grande o con mayor variabilidad en las consultas